# Handout 18

# **Semáforos II e Modelos de Concorrência**

## **1. Revisão: Semáforos e Rendez-vous**

Na aula anterior, exploramos o conceito de semáforos como mecanismos de sincronização que controlam o acesso de múltiplas threads a recursos compartilhados. Um exemplo prático foi a implementação de um *rendez-vous*, onde duas threads sincronizam suas execuções em pontos específicos do código.

**Exemplo de Rendez-vous com Semáforos:**


In [ ]:
#include <pthread.h>
#include <semaphore.h>
#include <stdio.h>

sem_t sem_A;
sem_t sem_B;

void* thread_A(void* arg) {
    printf("Thread A: Executando A1\n");
    sem_post(&sem_A); // Sinaliza que A1 foi concluído
    sem_wait(&sem_B); // Espera B1 ser concluído
    printf("Thread A: Executando A2\n");
    return NULL;
}

void* thread_B(void* arg) {
    printf("Thread B: Executando B1\n");
    sem_post(&sem_B); // Sinaliza que B1 foi concluído
    sem_wait(&sem_A); // Espera A1 ser concluído
    printf("Thread B: Executando B2\n");
    return NULL;
}

int main() {
    pthread_t tA, tB;

    sem_init(&sem_A, 0, 0);
    sem_init(&sem_B, 0, 0);

    pthread_create(&tA, NULL, thread_A, NULL);
    pthread_create(&tB, NULL, thread_B, NULL);

    pthread_join(tA, NULL);
    pthread_join(tB, NULL);

    sem_destroy(&sem_A);
    sem_destroy(&sem_B);

    return 0;
}

**Explicação:**

- sem_init: Inicializa os semáforos sem_A e sem_B com valor inicial 0.
- sem_post: Incrementa o semáforo, sinalizando que uma etapa foi concluída.
- sem_wait: Decrementa o semáforo; se o valor for zero, a thread é bloqueada até que o semáforo seja incrementado por outra thread.
- sem_destroy: Libera os recursos alocados para os semáforos.

## **2. Aplicações de Semáforos**

Semáforos são fundamentais para resolver problemas clássicos de sincronização em sistemas concorrentes. Um exemplo é o problema do Produtor-Consumidor, onde produtores geram dados que são consumidos por consumidores.

### **Problema Produtor-Consumidor com Buffer Limitado**

Neste problema, um ou mais produtores inserem itens em um buffer compartilhado, enquanto um ou mais consumidores retiram itens desse buffer. O desafio é garantir que:
- **Produtores** não insiram itens em um buffer cheio.
- **Consumidores** não retirem itens de um buffer vazio.

### **Implementação com Semáforos**
- **Semáforo `empty`:** Conta o número de posições vazias no buffer.
- **Semáforo `full`:** Conta o número de posições ocupadas no buffer.
- **Mutex `mutex`:** Garante acesso exclusivo ao buffer durante operações de inserção e remoção.
  
Os semáforos e o mutex ajudam a controlar o acesso ao buffer de forma segura e eficiente.

Implementação com Semáforos


In [ ]:
#include <pthread.h>
#include <semaphore.h>
#include <stdio.h>
#include <stdlib.h>

#define BUFFER_SIZE 5

int buffer[BUFFER_SIZE];
int in = 0;
int out = 0;

sem_t empty;
sem_t full;
pthread_mutex_t mutex;

void* producer(void* arg) {
    int item;
    while (1) {
        item = rand() % 100; // Produz um item
        sem_wait(&empty); // Decrementa o semáforo empty
        pthread_mutex_lock(&mutex); // Entra na seção crítica
        buffer[in] = item;
        in = (in + 1) % BUFFER_SIZE;
        printf("Produziu: %d\n", item);
        pthread_mutex_unlock(&mutex); // Sai da seção crítica
        sem_post(&full); // Incrementa o semáforo full
        sleep(1);
    }
}

void* consumer(void* arg) {
    int item;
    while (1) {
        sem_wait(&full); // Decrementa o semáforo full
        pthread_mutex_lock(&mutex); // Entra na seção crítica
        item = buffer[out];
        out = (out + 1) % BUFFER_SIZE;
        printf("Consumiu: %d\n", item);
        pthread_mutex_unlock(&mutex); // Sai da seção crítica
        sem_post(&empty); // Incrementa o semáforo empty
        sleep(1);
    }
}

int main() {
    pthread_t prod, cons;

    sem_init(&empty, 0, BUFFER_SIZE);
    sem_init(&full, 0, 0);
    pthread_mutex_init(&mutex, NULL);

    pthread_create(&prod, NULL, producer, NULL);
    pthread_create(&cons, NULL, consumer, NULL);

    pthread_join(prod, NULL);
    pthread_join(cons, NULL);

    sem_destroy(&empty);
    sem_destroy(&full);
    pthread_mutex_destroy(&mutex);

    return 0;
}


**Explicação:**

- Semáforo empty: Conta o número de posições vazias no buffer.
- Semáforo full: Conta o número de posições ocupadas no buffer.
- Mutex mutex: Garante acesso exclusivo ao buffer durante operações de inserção e remoção.
- Produtor:
    - Espera por uma posição vazia (sem_wait(&empty)).
    - Insere um item no buffer.
    - Sinaliza que há um item disponível (sem_post(&full)).
- Consumidor:
    - Espera por um item disponível (sem_wait(&full)).
    - Remove um item do buffer.
    - Sinaliza que há uma posição vazia (sem_post(&empty)).


## **3. Modelos de Concorrência**

Modelos de concorrência são abstrações que descrevem como múltiplas unidades de execução (threads ou processos) interagem e se sincronizam. Compreender esses modelos é essencial para projetar sistemas concorrentes eficientes e corretos.

### **3.1. Modelo de Memória Compartilhada**

Neste modelo, múltiplas threads ou processos compartilham o mesmo espaço de memória e comunicam-se através de variáveis compartilhadas.

**Vantagens:**
- Comunicação rápida devido ao acesso direto à memória.

**Desvantagens:**
- Necessidade de mecanismos de sincronização (como mutexes e semáforos) para evitar condições de corrida.

**Exemplo:**  
O exemplo do Produtor-Consumidor utiliza memória compartilhada para o buffer e semáforos para sincronização.

---

### **3.2. Modelo de Passagem de Mensagens**

Neste modelo, threads ou processos comunicam-se enviando e recebendo mensagens explícitas, sem compartilhar memória.

**Vantagens:**
- Evita problemas de sincronização relacionados à memória compartilhada.

**Desvantagens:**
- Pode introduzir sobrecarga devido à necessidade de copiar dados nas mensagens.

**Exemplo:**  
Sistemas distribuídos frequentemente utilizam o modelo de passagem de mensagens para comunicação entre processos em diferentes máquinas.